In [17]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import json
from tqdm import tqdm
from sklearn.metrics import average_precision_score

In [4]:
sl_path = '/home/jasonclarke/H-drive/mnt/parscratch/users/acp21jrc/sl_ASD_github/outputs/pyannote_MD-0.0_visible/'
light_path = '/home/jasonclarke/H-drive/users/acp21jrc/Light-ASD_Ego4D/output/results/'

# collect all the json files
json_files_light = {f.split('/')[-1].split('.')[0]: f for f in glob(light_path + '/*.json')}

# find all the json files that are not in the sl path
json_files_sl = {f.split('/')[-1].split('.')[0]: f for f in glob(sl_path + '/*.json')}



In [36]:
import numpy as np
from sklearn.metrics import average_precision_score

# Initialize arrays with efficient datatypes
scores = np.array([], dtype=np.float32)  # Halves memory usage vs float64
labels = np.array([], dtype=np.uint8)    # 8x more efficient than float64

for trackid in tqdm(json_files_light):
    with open(json_files_light[trackid]) as f:
        light_data = json.load(f)

    if trackid in json_files_sl:
        with open(json_files_sl[trackid]) as f:
            sl_data = json.load(f)
        
        assert len(sl_data) == len(light_data), f"Length mismatch for {trackid}"

        # Preallocate temporary arrays
        track_scores = np.zeros(len(light_data), dtype=np.float32)
        track_labels = np.zeros(len(light_data), dtype=np.uint8)

        for i, (frame_light, frame_sl) in enumerate(zip(light_data, sl_data)):
            track_scores[i] = (float(frame_light['score']) + float(frame_sl['score'])) / 2
            # track_scores[i] = float(frame_sl['score'])  # Use only light scores
            track_labels[i] = int(round(frame_light['label']))  # Ensure binary labels

        scores = np.concatenate([scores, track_scores])
        labels = np.concatenate([labels, track_labels])
    else:
        track_scores = np.zeros(len(light_data), dtype=np.float32)
        track_labels = np.zeros(len(light_data), dtype=np.uint8)

        for i, frame_light in enumerate(light_data):
            track_scores[i] = float(frame_light['score'])
            track_labels[i] = int(round(frame_light['label']))

        scores = np.concatenate([scores, track_scores])
        labels = np.concatenate([labels, track_labels])

ap_score = average_precision_score(labels, scores)

100%|██████████| 4068/4068 [01:59<00:00, 33.92it/s]


In [35]:
ap_score

np.float64(0.5570521870340842)

In [22]:
print(len(json_files_light))

4068


In [21]:
len(labels)

472291